In [124]:
import numpy as np

In [125]:
# Number of qubits.
N = 4

L = N // 2 # Length of half cut number of qubits.

'''
Intiating a wave function with a lsit of size 2**N with all element as zeros.

''' 

Psi_List = [0]*(2**N)

In [126]:
'''
Enter the non-zero coefficients of the wavefunction psi.

''' 

Psi_List[0] = 2/np.sqrt(13)
Psi_List[2] = np.sqrt(2/13)
Psi_List[4] = -1/np.sqrt(13)
Psi_List[14] = np.sqrt(6/13)

Let $\Psi$ be the given normalized wave function. Then $\rho_{AB} = <\Psi|\Psi>$ is the density matrix of the subsystem $A$ and $B$, and $\rho_{A} = Tr_{B}(\rho_{AB})$.
The $(s,s^{'})$ element of the reduced density matrix $\rho_{A}$ is given by 
$$<s|\rho_{A}|s^{'}> =\sum_{s^{''}} <ss^{''}|\rho_{AB}|s^{'}s^{''}>$$
$$<s|\rho_{A}|s^{'}> =\sum_{s^{''}} <ss^{''}|\psi><\psi|s^{'}s^{''}>$$

In [127]:
'''
It is not necessary to input a normalized wave function. 
It will be normalized later so that trace(rho) = 1.

'''

# Converting the list to a numpy matrix.
Psi = np.matrix(Psi_List).reshape(len(Psi),1) # Psi column matrix.

# Normalizing Psi.
Psi = Psi/np.linalg.norm(Psi)



$$\psi_{s^{'}} = \Psi[ 2^{L} s^{'} : 2^{L} s^{'}+2^{L}-1] $$

In [128]:
def psi(s):
    return Psi[(2**L)*s:(2**L)*s + 2**L]

The elements of the matrix $\rho_{A}$ is given by
$$<s|\rho_{A}|s^{'}> = \psi^{\dagger}_{s^{'}} \psi_{s}$$

In [129]:
'''
    psi(s_p) is a row matrix/vector. psi(s) is a column matrix/vector.      
    Dimension of rhoA is N/2 x N/2. 
    The element <s|rhoA|sp> is given by psi_sp^\dagger * psi_s.
''' 

def rhoA(s,s_p): # <s|rho_A|s_p>
    


    # psi(s_p)^\dagger * psi(s) is the element of (s,s_p) of rho_AB.  
    return psi(s_p).getH() * psi(s)

In [130]:
def rhoA_Matrix(N):
    
    M = np.zeros((N,N)) # 0 to N-1.
    
    '''
    rho is Hermitian, it is sufficient to calculate the elements above the diagonal.
    The the elements below the diagonal can be replace by the complex cpnjugate of the
    elements above the diagonal.
    '''
    for i in range(N):
        for j in range(N):
            
            if i <= j : # Above the diagonal (i,j) i<j.
                
                M[i,j] = rhoA(i,j)[0,0]
                
            else: # Below the diagonal (i,j) i>j.
                
                M[i,j] = np.conjugate(M[j,i])
    return M

In [131]:
'''
w is the diagonal of the diagonalized matrix rhoA.

'''
w, v = np.linalg.eig(rhoA_Matrix(N))

In [132]:
DL = np.zeros(N) # Creating an array for log w with zeros.

'''
The following loop calculates S = - sum \lamba_i * log(\lambda_i).

'''

for i in range(len(w)):
    
    if abs(w[i]) < 1.e-8: # log of zero gives nan.
        
        pass # Leave the log(zero) element as zero.
    
    else:
        
        DL[i] = np.log(w[i])
        
# Entropy = -Tr(rho * log(rho)).        
print('Entropy S = ',-sum(w*DL))

Entropy S =  0.5663916641767007


## The rolling operator

In [163]:
def Bin2Dec(BinaryNumber): # Converts binary to decimal numbers.
    return int(str(BinaryNumber),2)

def Dec2Bin(DecimalNumber): # Converts decimal to binary numbers.
    return bin(DecimalNumber).replace("0b", "")

List = [i for i in range(2**N)]

# This function converts all decimals from 0 to 2^N -1 in List to binary.
def List_Bin(List):
    
    l = []
    
    for i in List:
        
        i_Bin = Dec2Bin(i)
              
        
        '''
        While converting numbers from decimal to binary, for example, 1 is mapped to 1, to make sure that
        every numbers have N qubits in them, the following loop adds leading zeros to make the
        length of the binary string equal to N. Now, 1 is mapped to 000.....1 (string of length N).
        
        '''
        
        while len(i_Bin) < N: 
            
            i_Bin = '0'+i_Bin # This loop adds leading zeros.
            
        l.append(i_Bin)
        
    return l

'''
The following loop takes a binary string as input and rolls the qubits by one and returns the rolled string.

'''
def Roll_String(Binary_String):
    
    return Binary_String[-1] + Binary_String[:-1]


def Psi_Rolled(Inital_Psi):
    
    l = [Roll_String(i) for i in List_Bin(List)] # Rolls every string in the list List by one qubit.

    l_d = [Bin2Dec(i) for i in l] # Converts the rolled binary string to decimal number.


    Psi_Rolled = []

    for i in range(2**N):
    
        Psi_Rolled.append(Inital_Psi[l_d[i]])
        
    return Psi_Rolled

def N_Rolled(N, Initial_Psi):
    
    s = Psi_Rolled(Initial_Psi)
    
    for i in range(N-1):
        
        s = Psi_Rolled(s)
        
    return s

In [179]:
N_Rolled(8, Psi_List)

[0.5547001962252291,
 0,
 0.3922322702763681,
 0,
 -0.2773500981126146,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0.6793662204867574,
 0]